## This is the scarping codes

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import pandas as pd
import os
import random
from bs4 import BeautifulSoup

In [ ]:
path = r'E:\UCL\BENV0114\CHINATOWN\alldata' ### customize this path to excute following codes

### firstly define functions which will be used in scraping.
### those are scraping for the whole comments and monthly comments seperately

In [2]:
def scrape(url,driver,num_refresh):
    driver.get(url = url)

    body = driver.find_element_by_xpath('/html')

    time.sleep(3+np.random.rand()*2)
    
    print('finding "see all reviews"')
    count = 0
    while 1:
        ### simulate user's behaviour to pagedown
        actions_pagedown = ActionChains(driver)
        actions_pagedown.move_to_element(body)
        actions_pagedown.send_keys(Keys.PAGE_DOWN)
        actions_pagedown.perform()
        time.sleep(0.5)
        try:
            ### click See all reviews
            fe = driver.find_element_by_xpath("//*[contains(text(), 'See all reviews')]")
            actions_click = ActionChains(driver)
            actions_click.move_to_element(fe)
            actions_click.click()
            actions_click.perform()
            print('successfully found')
            break
        except:
            if count >= 30:
                print('failed to find')
                break
            else:
                print('retry')
                count += 1
                continue
                
    ### simulate user's behaviour to load new lada
            
    pane = driver.find_element_by_xpath('//*[@id="pane"]/div')
    
    actions_tab = ActionChains(driver)
    actions_tab.move_to_element(pane)
    actions_tab.send_keys(Keys.TAB)
    actions_tab.send_keys(Keys.TAB)
    actions_tab.perform()
    actions_pagedown_ = ActionChains(driver)
    actions_pagedown_.send_keys(Keys.PAGE_DOWN)
    
    print('loading data')
    ### randomly pause to avoid to be detected as a robot
    for i in range(num_refresh):
        dice = np.random.rand()
        time.sleep(dice*3)
        actions_pagedown_.perform()
        if dice <0.2:
            time.sleep(dice*random.choice(range(4,8)))
    
    return driver.page_source # return the html file to be decoded by the next function

def decode_html(page_source):
    soup = BeautifulSoup(page_source,'html.parser')
    
    res_stars = soup.find_all(class_ = 'section-review-stars')#find ratings
    
    res = soup.find_all(class_ = 'section-review-text')#find comments
    
    data = pd.DataFrame(columns=['text','label'])

    text = []
    
    jstcache = res[0].attrs['jstcache'] 
    for i in res:

        if i.attrs['jstcache'] == jstcache: ### avoid save replies as comments
            
            if len(i.contents) > 0:
                text.append(i.contents[0])
            else:text.append(' ')

    labels = []
    for i in res_stars:
        labels.append(i.attrs['aria-label'][1])

    assert(len(text)==len(labels)) ### ensure the length of comments and ratings are matched

    data.loc[:,'text'] = text

    data.loc[:,'label'] = labels
        
    return data

#### functions below are scraping for monthly comments, but the logics are the same

def scrape_month(url,driver,pi):

    driver.get(url = url)

    body = driver.find_element_by_xpath('/html')

    time.sleep(3+np.random.rand()*2)

    print('finding "see all reviews"')
    count = 0
    while 1:
        actions_pagedown = ActionChains(driver)
        actions_pagedown.move_to_element(body)
        actions_pagedown.send_keys(Keys.PAGE_DOWN)
        actions_pagedown.perform()
        time.sleep(0.5)
        try:
            fe = driver.find_element_by_xpath("//*[contains(text(), 'See all reviews')]")
            actions_click = ActionChains(driver)
            actions_click.move_to_element(fe)
            actions_click.click()
            actions_click.perform()
            print('successfully found')
            break
        except:
            if count >= 3:
                print('failed to find')
                break
            else:
                print('retry')
                count += 1
                continue
    pane = driver.find_element_by_xpath('//*[@id="pane"]/div')
    time.sleep(0.2)
    driver.find_elements_by_xpath("//*[contains(text(), 'Sort')]")[0].click()
    time.sleep(0.2)
    driver.find_elements_by_xpath("//*[contains(text(), 'Newest')]")[0].click()
    
    actions_tab = ActionChains(driver)
    actions_tab.move_to_element(pane)
    actions_tab.send_keys(Keys.TAB)
    actions_tab.send_keys(Keys.TAB)
    actions_tab.perform()
    actions_pagedown_ = ActionChains(driver)
    actions_pagedown_.send_keys(Keys.PAGE_DOWN)
    
    print('loading data')
    while 1:
        dice = np.random.rand()
        time.sleep(dice*3)
        actions_pagedown_.perform()
        if dice <0.2:
            time.sleep(dice*random.choice(range(4,8)))
        

        soup = BeautifulSoup(driver.page_source,'html.parser')

        date = soup.find_all(class_ = 'section-review-publish-date')

        if ('months' or 'year' or 'years') in ' '.join([j for i in date for j in i.contents]):

            break
    return driver.page_source

def decode_html_date(page_source):
    soup = BeautifulSoup(page_source,'html.parser')
    
    res_stars = soup.find_all(class_ = 'section-review-stars')
    
    res = soup.find_all(class_ = 'section-review-text')
    
    date = soup.find_all(class_ = 'section-review-publish-date')
    
    data = pd.DataFrame(columns=['text','label','date'])

    text = []
    jstcache = res[0].attrs['jstcache']
    for i in res:

        if i.attrs['jstcache'] == jstcache:
            
            if len(i.contents) > 0:
                text.append(i.contents[0])
            else:text.append(' ')

    labels = []
    for i in res_stars:
        labels.append(i.attrs['aria-label'][1])
    
    date_list = []
    for i in date:
        date_list.append(i.contents[0])

    assert(len(text)==len(labels))

    data.loc[:,'text'] = text

    data.loc[:,'label'] = labels
    
    data.loc[:,'date'] = date_list
        
    return data

In [4]:
metadata = pd.read_csv(path+'\metadata.csv') # reading metadata

### define options and drivers

In [5]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})
driver = webdriver.Chrome(path+'\chromedriver79.exe',options = options)
driver.implicitly_wait(5)

### excecuting scarping for the whole comments

In [10]:
for i in range(metadata.shape[0]):
    if metadata.loc[i,'total_rating'] >= 5:
        name = metadata.loc[i,'name']
        place_id = metadata.loc[i,'place_id']
        url = 'https://www.google.com/maps/place/?q=place_id:{0}'.format(place_id)
        print('searching for {0}'.format(name))
        html = scrape_month(url,driver,place_id)
        print('decoding data')
        data = decode_html_date(html)
        data.to_csv(path+'\Monthly comments\{0}.csv'.format(name.replace('/','').replace('|','').replace('<','').replace('>','')),
                    index = False)

searching for Hong Ning Clinic 康宁中医诊所
finding "see all reviews"
successfully found
loading data
decoding data
searching for New China 中华楼
finding "see all reviews"
successfully found
loading data
decoding data
searching for HOT POT Thai Cuisine
finding "see all reviews"
successfully found
loading data
decoding data
searching for Shake Shack Leicester Square
finding "see all reviews"
successfully found
loading data
decoding data
searching for Canton
finding "see all reviews"
successfully found
loading data
decoding data
searching for MW Buffet
finding "see all reviews"
successfully found
loading data
decoding data
searching for LEON
finding "see all reviews"
successfully found
loading data
decoding data


KeyError: 118

### excecuting scraping for monthly data

In [6]:
for i in range(metadata.shape[0]):
    if metadata.loc[i,'total_rating'] >= 300:
        name = metadata.loc[i,'name']
        place_id = metadata.loc[i,'place_id']
        url = 'https://www.google.com/maps/place/?q=place_id:{0}'.format(place_id)
        print('searching for {0}'.format(name))
        html = scrape(url,driver,150)
        print('decoding data')
        data = decode_html(html)
        print('saving {0}'.format(name))
        data.to_csv(path+'\Extra_comments\{0}.csv'.format(name),
                index = False)

searching for Bun House
finding "see all reviews"
successfully found
loading data
decoding data
saving Bun House
searching for C & R Cafe Restaurant
finding "see all reviews"
successfully found
loading data
decoding data
saving C & R Cafe Restaurant
searching for Caffe Concerto - Shaftesbury Avenue
finding "see all reviews"
successfully found
loading data
decoding data
saving Caffe Concerto - Shaftesbury Avenue
searching for Cafe TPT
finding "see all reviews"
successfully found
loading data
decoding data
saving Cafe TPT
searching for Chinatown Bakery
finding "see all reviews"
successfully found
loading data
decoding data
saving Chinatown Bakery
searching for Chinatown Gate
finding "see all reviews"
successfully found
loading data
decoding data
saving Chinatown Gate
searching for Dumplings' Legend
finding "see all reviews"
successfully found
loading data
decoding data
saving Dumplings' Legend
searching for Doughnut Time
finding "see all reviews"
successfully found
loading data
decoding 

finding "see all reviews"
successfully found
loading data
decoding data
saving HOT POT Thai Cuisine
searching for Shake Shack Leicester Square
finding "see all reviews"
successfully found
loading data
decoding data
saving Shake Shack Leicester Square
searching for MW Buffet
finding "see all reviews"
successfully found
loading data
decoding data
saving MW Buffet
searching for LEON
finding "see all reviews"
successfully found
loading data
decoding data
saving LEON


In [7]:
driver.quit() ## quiting the scraping process